In [2]:
import pandas as pd 

In [6]:
df = pd.read_csv("Spotify Million Song Dataset_exported.csv")

In [7]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [8]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [9]:
df.head(10)

,artist,song,text
0,Styx,Why Me,I guess we used to be the lucky ones \nGood f...
1,Rolling Stones,Country Honk,I'm sittin' in a bar tipplin' a jar in Jackson...
2,Weird Al Yankovic,Llama Song,Here's a llama \nThere's a llama \nAnd anoth...
3,Alan Parsons Project,The Three Of Me,There's a voice on the phone \nWho just calle...
4,Marillion,Lavender,"(Derek dick, Steve Rothery, Mark Kelly, Pete T..."
5,Roy Orbison,Break My Mind,"Baby, oh, baby \n \nTell the man at the tick..."
6,Hanson,Save Me,Loving you like I never have before \nI'm nee...
7,Lucky Dube,Till You Lose It All,We take a lot of things for granted \nOne day...
8,America,Rainy Day,Whenever it's a rainy day \nI pack my trouble...
9,Faith Hill,O Come All Ye Faithful,"Oh, come ye \nOh come ye to Bethlehem \nCome..."


In [10]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
df.tail(5)

,artist,song,text
4995,Point Of Grace,Do It Again,just when i think it couldn't be any harder ...
4996,Frankie Valli,The Sun Ain't Gonna Shine Anymore,loneliness is the coat you wear a deep shade...
4997,Fifth Harmony,Leave My Heart Out Of This,my heart is telling me that you were gonna cha...
4998,John Prine,We Could,if anyone could find the joy that true love ...
4999,Scorpions,Deep And Dark,"down, i am down on the ground, sad and tired..."


In [14]:
import nltk
from nltk.stem.porter import PorterStemmer

In [23]:
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [24]:
tokenization('you are beautifull')

'you are beautiful'

In [17]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [29]:
similarity[0]

array([1.        , 0.00426294, 0.00756886, ..., 0.09684561, 0.07311986,
       0.02811694])

In [30]:
df[df['song'] == 'Crying Over You']

,artist,song,text
4291,UB40,Crying Over You,cri over you in the morn cri over you in the e...
4632,ABBA,Crying Over You,i 'm waitin ' for you babi i 'm sit all alon i...


In [31]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [32]:
recommendation('Crying Over You')

["I'm Gonna Cry (Cry Baby)",
 'Crying',
 "Don't Cry",
 "My Eyes Don't Cry",
 "Don't Cry Sister",
 'Cry Me A River',
 'Crying',
 'Breathe A Sigh',
 "Don't Cry Baby",
 'You Made Me Love You',
 'Place To Hide And Cry',
 "Don't Cry No More",
 'Cry',
 "Heroes Don't Cry",
 'I Wish It Would Rain',
 'Somewhere Tonight',
 "I'm So Lonesome I Could Cry",
 'I Cried Again',
 'WHY SHOULD I CRY',
 'Everyday']

In [33]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))